# Apartment number

In [ ]:
import folium
import geopandas as gpd
import pandas as pd
from libpysal import graph

In [ ]:
cd /home/lisa/work/people_places_germany/code

In [ ]:
from clusters import assign_clusters
from spatial_autocorrelation import lisa

In [ ]:
clusters = gpd.read_parquet(
    "/data/cluster_data/clusters_umap_freiburg_100_3_gaussian_euclidean_complete_chebyshev_3.pq"
)

In [ ]:
# clusters.explore(column=clusters.index,cmap='tab20')

# Raw Data

In [ ]:
demographics = pd.read_parquet("/data/processed_data/Geb100m.parquet").reset_index()

In [ ]:
gdf, largest_overlap, data = assign_clusters(demographics, clusters)

In [ ]:
gdf["STATS", "Apartment no"] = (
    data["APARTMENT_NO"]["1 apartment"].fillna(0) * 1
    + data["APARTMENT_NO"]["2 apartments"].fillna(0) * 2
    + data["APARTMENT_NO"]["3 - 6 apartments"].fillna(0) * 4.5
    + data["APARTMENT_NO"]["7 - 12 apartments"].fillna(0) * 9.5
    + data["APARTMENT_NO"]["13 and more apartments"].fillna(0) * 15
) / data["APARTMENT_NO"].sum(axis=1)

In [ ]:
gdf

In [ ]:
gdf1 = gdf[["ID", "geometry", "STATS", "cluster"]]
gdf1.columns = gdf1.columns.get_level_values(0)

In [ ]:
gdf2 = gdf[["ID", "APARTMENT_NO"]]
gdf2.columns = gdf2.columns.get_level_values(1)
gdf1 = gdf1.merge(gdf2, how="left", on="ID")

# Compute Spatial Lag

In [ ]:
gdf1 = gdf1[gdf1.cluster.notna()]
queen = graph.Graph.build_contiguity(gdf1, rook=False)
row_wise_queen = queen.transform("R")

In [ ]:
gdf1.columns

In [ ]:
for i in [
    "STATS",
    "1 apartment",
    "3 - 6 apartments",
    "2 apartments",
    "7 - 12 apartments",
    "13 and more apartments",
]:
    gdf1.loc[:, i + "_lag"] = row_wise_queen.lag(gdf1[i])

In [ ]:
gdf1.to_parquet(
    "/home/lisa/work/people_places_germany/Notebooks/temp_data/apartmentno.parquet"
)

In [ ]:
gdf1

In [ ]:
# gdf1.cx[4150000:4170000,2760000:2780000].explore("STATS", cmap="coolwarm", prefer_canvas=True)

# Local Spatial Autocorrelation

In [ ]:
mi, gdf_05 = lisa(gdf1.cx[4150000:4170000, 2760000:2780000], "STATS", 0.05)

In [ ]:
print(f"Moran's I: {mi.I}, p-value: {mi.p_sim}")

In [ ]:
# gdf_05.explore("cluster", prefer_canvas=True, cmap=["#d7191c","#fdae61","#abd9e9","#2c7bb6","lightgrey"])

# Cluster Aggregation

In [ ]:
d = []
for i in range(0, int(gdf1["cluster"].max())):
    d.append(
        {
            "stats": gdf1[gdf1["cluster"] == i]["STATS"].mean(),
            "stats count": gdf1[gdf1["cluster"] == i]["STATS"].count(),
        }
    )

stats = pd.DataFrame(d)

In [ ]:
stats["label"] = stats.index
stats = clusters.merge(stats, on="label")

In [ ]:
# stats.explore(column='stats', cmap="coolwarm", prefer_canvas=True)

In [ ]:
m = stats.explore(
    column="stats", cmap="coolwarm", prefer_canvas=True, name="cluster aggregation"
)

gdf_05.explore(
    "sig_cluster",
    prefer_canvas=True,
    cmap=["#d7191c", "#fdae61", "#abd9e9", "#2c7bb6", "lightgrey"],
    name="local spatial autocorrelation",
    m=m,
)

gdf1.cx[4150000:4170000, 2760000:2780000].explore(
    "STATS", cmap="coolwarm", prefer_canvas=True, name="raw data", m=m
)

# Add a layer control to toggle the layers on and off
folium.LayerControl().add_to(m)

# Display the map
m